In [ ]:
import requests

# API token for Songkick
# LA=17835, SF=26330, Chicago=9426, SD=11086, CIN=22040
rid = '22040'
per_page = 50
token = 'WGLTwENb36bAgHNc'
query_params = { 'apikey': token,
                 'location': 'sk:'+rid,
                 'per_page': per_page,
                 'page': 1,
         }

#endpoint = 'http://api.songkick.com/api/3.0/venues/'+venueId+'/calendar.json?'
endpoint = 'http://api.songkick.com/api/3.0/events.json?'

# One preliminary call to get the total number of entries for pagination:
response = requests.get(endpoint, params=query_params)
total_results = response.json()['resultsPage']['totalEntries']
print "Total number of entries:", total_results
total_calls = int(total_results/per_page) # Need at least this many, plus:
# If 151, or 242, need one more call to grab remainder
if float(total_results)/per_page - int(total_results/per_page) > 0:
    total_calls += 1
    

# Loop through all pages, collect 50 events/page
all_events_raw = []
for p in range(total_calls):
    query_params['page'] = (p+1)
    response = requests.get(endpoint, params=query_params)
    full_data = response.json()['resultsPage']['results']['event']
    all_events_raw.append(full_data)
    if p % 20 == 0:
        print "Made %i calls, corresponding to %i events..." % (p, p*50)

# Sanity check
print "Done!"
print "Found this many pages of venues:", len(all_events_raw)


In [ ]:
# Sanity checks here, optional cell
for page in all_events_raw:
    for evt in page:
        print evt['location']['city']
        print evt['venue']['metroArea']['id']


In [ ]:

import pymysql as mdb
con = mdb.connect('localhost', 'root', '', 'scenehash', autocommit=True, charset='utf8', use_unicode=True) 
cur = con.cursor()
'''
cur.execute("DROP TABLE IF EXISTS events")
cur.execute("CREATE TABLE events"
            "(pid INT PRIMARY KEY AUTO_INCREMENT, "
            "evt_id INT, "
            "evt_name VARCHAR(400), "
            "venue_name VARCHAR(150), "
            "venue_id INT, "
            "city_name VARCHAR(50), "
            "metro_id INT, "
            "latitude VARCHAR(20), "
            "longitude VARCHAR(20), "
            "evt_url VARCHAR(200), "
            "artist VARCHAR(200), "
            "artist_id INT, "
            "artist_billing VARCHAR(100) ) "
            )
'''
new_inserts = 0
for evt_page in all_events_raw:
    for evt in evt_page:
        
        city = evt['location']['city']
        metroId = evt['venue']['metroArea']['id']
        # Skip if no artists
        if len(evt['performance']) == 0:
            continue
        lat = evt['location']['lat']
        lng = evt['location']['lng']
        # Skip if no lat/lon for venue
        url = evt['uri']
        venueId = evt['venue']['id']
        venueName = evt['venue']['displayName']
        evtName = evt['displayName']
        evtId = evt['id']
        
        cur.execute("SELECT * FROM events WHERE evt_id = %i" % evtId)
        rows = cur.fetchall()
        if not rows:

            for a in evt['performance']:
                #print a['artist']['displayName'], a['artist']['id'], a['billing']
                sql_str = 'INSERT INTO events (evt_id,evt_name,venue_name,venue_id,city_name,metro_id,latitude,longitude,evt_url,artist,artist_id,artist_billing) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
                sql_vals = ( evtId, evtName, venueName, venueId, city, metroId, lat, lng, url, a['artist']['displayName'], a['artist']['id'], a['billing'])
                try:
                    cur.execute(sql_str, sql_vals)
                except Exception:
                    continue
            print "Inserted:", evtName
            new_inserts += 1

print "Total new inserts =", new_inserts


In [ ]:

with con:
    cur = con.cursor()
    cur.execute("SELECT * FROM events")
    rows = cur.fetchall()
    for row in rows:
        print row
        

In [1]:
# Isolated cell that queries sql database and creates list of id/artist pairs
# Songkick id will be used in next cell against EchoNest API.

all_artists = []
import pymysql as mdb
con = mdb.connect('localhost', 'root', '', 'scenehash', autocommit=True, charset='utf8', use_unicode=True) 
cur = con.cursor()
with con:
    cur = con.cursor()
    cur.execute("SELECT artist_id, artist FROM events")
    rows = cur.fetchall()
    for row in rows:
        if row[0] not in all_artists:
            all_artists.append([row[0], row[1]])

# all_artist is a list of unique artist_ids
print len(all_artists)



19157


In [2]:
import requests
import time
echo_token = 'WL4JBSOI4ZGXPY8FN'

pars = { 'api_key': echo_token,
                 'name': 'songkick:artist:177850',
                 'bucket': 'hotttnesss',
                 'bucket': 'familiarity',
                 'bucket': 'terms',
         }

endpoint = 'http://developer.echonest.com/api/v4/artist/profile?'

artist_info_list = []
counter = 0
for artId, artist in all_artists[15000:]:
    # Echo Nest allows 120 calls/min, so:
    time.sleep(.5)
    # Progress report (before continue statemenst below, because we care about 
    # total number of API calls made)
    counter += 1
    if counter % 500 == 0:
        print "Made %i calls, corresponding to %i valid artist (%f)..." % (counter, len(artist_info_list), len(artist_info_list)/float(counter))
    # Set the parameter 'name' to current songkick id
    pars['name'] = 'songkick:artist:'+str(artId)
    # Make API call to Echo Nest
    response = requests.get(endpoint, params=pars)
    if 'artist' not in response.json()['response']:
        continue
    if not response.json()['response']['artist']['terms']:
        continue
    #print response.headers['X-RateLimit-Remaining']
    # We want [ id, name, {term1},{term2},{term3},... ]
    id_name_pair = [artId, artist]
    artist_info_list.append(id_name_pair + response.json()['response']['artist']['terms'])

print "Done!"
print "Total number of artists retained:", len(artist_info_list)

import pickle
pickle.dump( artist_info_list, open( "artist_info_15k_20k.pickle", "wb" ) )


Made 500 calls, corresponding to 423 valid artist (0.846000)...
Made 1000 calls, corresponding to 744 valid artist (0.744000)...
Made 1500 calls, corresponding to 1067 valid artist (0.711333)...
Made 2000 calls, corresponding to 1370 valid artist (0.685000)...
Made 2500 calls, corresponding to 1715 valid artist (0.686000)...
Made 3000 calls, corresponding to 2039 valid artist (0.679667)...
Made 3500 calls, corresponding to 2470 valid artist (0.705714)...
Made 4000 calls, corresponding to 2852 valid artist (0.713000)...
Done!
Total number of artists retained: 2984


In [ ]:
for i in artist_info_list:
    print i
print len(artist_info_list)
